### 1. 服务场景介绍
假设咱们现在开发一个电商网站，要实现支付订单的功能，流程如下：
* 创建一个订单之后，如果用户立刻支付了这个订单，我们需要将订单状态更新为“已支付”
* 扣减相应的商品库存
* 通知仓储中心，进行发货
* 给用户的这次购物增加相应的积分  

针对上述流程，我们需要有订单服务、库存服务、仓储服务、积分服务。整个流程的大体思路如下：
* 用户针对一个订单完成支付之后，就会去找订单服务，更新订单状态
* 订单服务调用库存服务，完成相应功能
* 订单服务调用仓储服务，完成相应功能
* 订单服务调用积分服务，完成相应功能  

至此，整个支付订单的业务流程结束

### 2. 微服务下要解决的问题
* **服务注册与发现**  
  负责得知每个微服务组件的地址和端口  
    * 咱们来考虑第一个问题：订单服务想要调用库存服务、仓储服务，或者是积分服务，怎么调用？订单服务压根儿就不知道人家库存服务在哪台机器上啊！他就算想要发起一个请求，都不知道发送给谁，有心无力！  
    * 这时候，就轮到Spring Cloud Eureka出场了。Eureka是微服务架构中的注册中心，专门负责服务的注册与发现。   
* **通用的构造request,解析response的功能**  
   简化发起request, 解析response的请求; 将服务调用方式解耦: http, rpc等  
* **负责对访问进行负载均衡**  
    * 一个微服务通常部署在多台机器上分散每台机器的并发访问压力. 因此服务注册于发现那一步中, 通常会给每个微服务注册多个地址; 此时, 就应该改采用负载均衡的办法从中选出一个地址用来调用.
    * 一般可以使用的最经典的Round Robin轮询算法。这是啥？简单来说，就是如果订单服务对库存服务发起10次请求，那就先让你请求第1台机器、然后是第2台机器、第3台机器、第4台机器、第5台机器，接着再来—个循环，第1台机器、第2台机器。。。以此类推。
* **雪崩:对某个微服务的隔离,熔断和降级**    
   * 在微服务架构里，系统有时会在在一个操作内调用多个的服务, 这么多服务互相调用，要是不做任何保护的话，某一个服务挂了，就会引起连锁反应，导致别的服务也挂。   
     以本文的业务场景为例：订单服务在一个业务流程里需要调用三个服务。现在假设订单服务自己最多只有100个线程可以处理请求，然后呢，积分服务不幸的挂了，每次订单服务调用积分服务的时候，都会卡住几秒钟，然后抛出—个超时异常。
   * 隔离:   
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;意思是说搞很多个小小的线程池，比如订单服务请求库存服务是一个线程池，请求仓储服务是一个线程池，请求积分服务是一个线程池。每个线程池里的线程就仅仅用于请求那个服务。  
     &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;打个比方：现在很不幸，积分服务挂了，会咋样？  
     &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;当然会导致订单服务里的那个用来调用积分服务的线程都卡死不能工作了啊！但是由于订单服务调用库存服务、仓储服务的这两个线程池都是正常工作的，所以这两个服务不会受到任何影响。  
     &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;这个时候如果别人请求订单服务，订单服务还是可以正常调用库存服务扣减库存，调用仓储服务通知发货。只不过调用积分服务的时候，每次都会报错。但是如果积分服务都挂了，每次调用都要去卡住几秒钟干啥呢？有意义吗？当然没有！所以我们直接对积分服务熔断不就得了，比如在5分钟内请求积分服务直接就返回了，不要去走网络请求卡住几秒钟，这个过程，就是所谓的熔断！  
   * 降级:   
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;当某个微服务宕机后, 本该调用改为服务的流程改为一些低成本的操作, 比如在数据库中记录下此次操作的计划  
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;回到本例, 积分服务挂了你就熔断，有的兄弟会问, 好歹你干点儿什么啊！别啥都不干就直接返回啊？没问题，咱们就来个降级：每次调用积分服务，你就在数据库里记录一条消息，说给某某用户增加了多少积分，因为积分服务挂了，导致没增加成功！这样等积分服务恢复了，你可以根据这些记录手工加一下积分。这个过程，就是所谓的降级。
* **微服务网关** 
    * 如果前端、移动端要调用后端系统，统一从网关进入，由网关转发请求给对应的服务      
    * 假设你后台部署了几百个服务，现在有个前端兄弟，人家请求是直接从浏览器那儿发过来的。打个比方：人家要请求一下库存服务，你难道还让人家记着这服务的名字叫做inventory-service？部署在5台机器上？就算人家肯记住这一个，你后台可有几百个服务的名称和地址呢？难不成人家请求一个，就得记住一个？你要这样玩儿，那真是友谊的小船，说翻就翻！    
      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;上面这种情况，压根儿是不现实的。所以一般微服务架构中都必然会设计一个网关在里面，像android、ios、pc前端、微信小程序、H5等等，不用去关心后端有几百个服务，就知道有一个网关，所有请求都往网关走，网关会根据请求中的一些特征，将请求转发给后端的各个服务。    
       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;而且有一个网关之后，还有很多好处，比如可以做统一的降级、限流、认证授权、安全，等等。